# Auto Correct

In this notebook I want to implement a fully working auto correct system which is simple yet functional. To elaborate, the system would correct words that are 1 and 2 edit distances away. 
An edit could consist of one of the following:
- Delete: "mangoz" -> "mango", "mngoz", ...

- Switch: "atm": "tam", "amt", ...

- Replace: "rat": "hat", "mat", "rap", ...

- Insert: "pla": "play", "pela", ...

# Data PreProcessing


In [6]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import random

The first step is to define a __process_data__ function that would read the corpus --*shakespeare.txt*-- and build a vocabulary based on that.

In [40]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You            just have to read it in. 
    Output: 
        words: a list containing all the  words in the corpus (text           file you read) in lower case. 
    """
    with open(file_name) as f:
        raw = f.read()
    low_data = raw.lower()
    words = re.findall(r"\w+", low_data)
    
    return words

In [41]:
words = process_data("shakespeare.txt")
vocab = set(words)
print(f"There are {len(vocab)} unique words. \nHere's 10 random samples from the vocabulary:")
random.sample(vocab, 10)

There are 6116 unique words. 
Here's 10 random samples from the vocabulary:


['know',
 'parcels',
 'pain',
 'o',
 'despair',
 'define',
 'submit',
 'grounded',
 'tyrants',
 'hercules']

Now, we need a dictionary with words as its keys and the value of each is the number of times that word appear in our corpus. A word_frequency dictionary in essence. 

In [42]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the           word and value is its frequency.
    '''
    
    word_count_dict = {} 
    for word in word_l:
        
        if word_count_dict.get(word, False):
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1

    return word_count_dict

In [44]:
word_frequency = get_count(words)
sample_words = random.sample(list(word_frequency),10)
{k: word_frequency[k] for k in sample_words} ## 10 random sample from word_frequency dictionary

{'kindness': 2,
 'vanquish': 1,
 'underneath': 1,
 'ruinate': 1,
 'serpent': 3,
 'prescriptions': 2,
 'expressing': 1,
 'chiding': 1,
 'wife': 41,
 'open': 5}

In [48]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the           word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values          are the probability that a word will occur. 
    '''
    
    total_words = sum(word_count_dict.values())
    probs = {word: word_count_dict[word]/total_words for word in word_count_dict.keys()}
    
    return probs

In [49]:
probs = get_probs(word_frequency)

# String Edits

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [50]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all                possible words in the vocabulary which have 1 missing                character
    Output:
        delete_l: a list of all possible strings obtained by                 deleting 1 character from word
    '''
     
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [L+R[1:] for L,R in split_l if R]


    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [51]:
delete_word_l = delete_letter(word="brand",
                        verbose=True)

input word brand, 
split_l = [('', 'brand'), ('b', 'rand'), ('br', 'and'), ('bra', 'nd'), ('bran', 'd'), ('brand', '')], 
delete_l = ['rand', 'band', 'brnd', 'brad', 'bran']


In [52]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent           charater switched
    ''' 
    
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [L[:-1] + R[0] + L[-1] + R[1:] for L,R in split_l if R and L]

    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [53]:
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [55]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced           one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_l = [L + let + R[1:] for L,R in split_l for let in letters]
    replace_set = set(replace_l)
    ### START CODE HERE ###

    ### END CODE HERE ###
    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [56]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'can', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [57]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter           inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [L + let + R for L,R in split_l for let in letters]

    ### START CODE HERE ###

    ### END CODE HERE ###

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [58]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


# Combining Edits
We need two functions that wrap up all the possible single and double edits. `edit_one_letter()` and `edit_two_letters()`

## Reference 
NLP Specialization course from Stanford university